In [1]:
!pip install -q prettytable

In [2]:
from prettytable import PrettyTable


def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [3]:
import warnings

warnings.filterwarnings("ignore")
# onnx for network architecture visualization
import onnx
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt

In [4]:
from JigsawNet import JigsawNet

model = JigsawNet(n_classes=50, num_features=3072, relu_in_last_fc=True)
count_parameters(model)

+------------+------------+
|  Modules   | Parameters |
+------------+------------+
| fc1.weight |  1572864   |
|  fc1.bias  |    512     |
| fc2.weight | 301989888  |
|  fc2.bias  |   16384    |
| fc3.weight |  67108864  |
|  fc3.bias  |    4096    |
| fc4.weight |   204800   |
|  fc4.bias  |     50     |
| bn4.weight |     50     |
|  bn4.bias  |     50     |
+------------+------------+
Total Trainable Params: 370897558


370897558

In [5]:
dummy_input = torch.randn(1, 36, 3072)
input_names = ["dummy_input"]
output_names = ["dummy_output"]

In [6]:
onnx_path = "results/onnx_JigsawNet.pt"
torch.onnx.export(
    model, dummy_input, onnx_path, input_names=input_names, output_names=output_names
)
onnx.save(onnx.shape_inference.infer_shapes(onnx.load(onnx_path)), onnx_path)
# Then go to netron.app in your browser and choose the exported file to visualize

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [7]:
# code from TA
import torch.nn as nn
import torch.nn.functional as F


# # Define the model architecture
class JigsawModel(nn.Module):
    def __init__(self, num_positions):
        super(JigsawModel, self).__init__()
        self.num_positions = num_positions
        self.fc1 = nn.Linear(36 * 2048, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 50)
        self.bn4 = nn.BatchNorm1d(50)  # Batch normalization after fc4

    def forward(self, x):
        x = x.view(-1, 36 * 2048)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(
            self.bn4(self.fc4(x))
        )  # Apply batch normalization after fc4 and before activation
        x = F.softmax(x, dim=1)
        return x


model = JigsawModel(num_positions=50)
count_parameters(model)

+------------+------------+
|  Modules   | Parameters |
+------------+------------+
| fc1.weight |  75497472  |
|  fc1.bias  |    1024    |
| fc2.weight |   524288   |
|  fc2.bias  |    512     |
| fc3.weight |   65536    |
|  fc3.bias  |    128     |
| fc4.weight |    6400    |
|  fc4.bias  |     50     |
| bn4.weight |     50     |
|  bn4.bias  |     50     |
+------------+------------+
Total Trainable Params: 76095510


76095510

In [8]:
dummy_input = torch.randn(1, 36, 2048)
input_names = ["dummy_input"]
output_names = ["dummy_output"]

In [9]:
onnx_path = "results/onnx_JigsawModel.pt"
torch.onnx.export(
    model, dummy_input, onnx_path, input_names=input_names, output_names=output_names
)
onnx.save(onnx.shape_inference.infer_shapes(onnx.load(onnx_path)), onnx_path)
# Then go to netron.app in your browser and choose the exported file to visualize

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [10]:
from JigsawNet import JigsawNet

model = JigsawNet(
    n_classes=50, num_features=2048, relu_in_last_fc=True, include_softmax=True
)

onnx_path = "results/onnx_JigsawNet_2048.pt"
torch.onnx.export(
    model, dummy_input, onnx_path, input_names=input_names, output_names=output_names
)
onnx.save(onnx.shape_inference.infer_shapes(onnx.load(onnx_path)), onnx_path)
# Then go to netron.app in your browser and choose the exported file to visualize

pytorch_total_params = sum(p.numel() for p in model.parameters())

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [11]:
pytorch_total_params

370373270

In [12]:
count_parameters(model)

+------------+------------+
|  Modules   | Parameters |
+------------+------------+
| fc1.weight |  1048576   |
|  fc1.bias  |    512     |
| fc2.weight | 301989888  |
|  fc2.bias  |   16384    |
| fc3.weight |  67108864  |
|  fc3.bias  |    4096    |
| fc4.weight |   204800   |
|  fc4.bias  |     50     |
| bn4.weight |     50     |
|  bn4.bias  |     50     |
+------------+------------+
Total Trainable Params: 370373270


370373270

## Bigger Model without Softmax Doesn't Work

In [1]:
import pandas as pd


def calc_accuracy(ground_truth_file, result_file):
    groud_truth = pd.read_csv(ground_truth_file, header=None)
    result = pd.read_csv(result_file, header=None)
    acc = 1 - len(result.compare(groud_truth)) / len(groud_truth)
    return acc

In [10]:
acc = calc_accuracy("../cs712-phuong/ensemble_test.txt", "./data/test.txt")
acc

0.31920710868079294

## Use Phuong's Methodology


In [5]:
%%time

!python train_v9.py > logs/v9-e-300-b-16-f-20-lr-0.0001-step-182-train.txt

CPU times: user 1.16 s, sys: 284 ms, total: 1.45 s
Wall time: 1min 41s


In [6]:
%%time

!python validate-timm.py -n test > logs/v9-e-300-b-16-f-20-lr-0.0001-step-182-test.txt

CPU times: user 52.3 ms, sys: 19.6 ms, total: 71.9 ms
Wall time: 5 s


In [8]:
acc = calc_accuracy("./results/ensemble_test.txt", "./data/test.txt")
acc

0.9432672590567327

In [9]:
acc = calc_accuracy("./results/test.txt", "./data/test.txt")
acc

0.4313055365686944

In [12]:
%%time

!python validate-timm.py > logs/v9-e-300-b-16-f-20-lr-0.0001-step-182-val.txt

CPU times: user 67.8 ms, sys: 692 µs, total: 68.5 ms
Wall time: 4.82 s


In [13]:
acc = calc_accuracy("./results/validation.txt", "./data/validation.txt")
acc

0.44542974079126874